In [ ]:
#This notebook should run on Google Colab
#Run this section on Colab, and then restart runtime
!sudo apt-get install build-essential swig 
!pip install auto-sklearn==0.11.1

In [1]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import numpy as np

## frequency of monthly wildfire by states ##

In [2]:
#The needed csv should be in google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/733project/etl_filtered_v1.csv')
df['month'] = pd.DatetimeIndex(df['datetime']).month
df

,Unnamed: 0,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,CONT_DATE,DISCOVERY_DATE,CONT_DOY,DISCOVERY_TIME,CONT_TIME,...,STATE,datetime,population_density,ELEVATION,AWND,PRCP,TMAX,TMIN,distance,month
0,0,2011,47,Debris Burning,5.0,2455608.5,2455608.5,47.0,1300.0,1530.0,...,AL,2011-02-16,7.572142,100.6,8.062714,0.0,71.000000,32.000000,0.141421,2
1,1,2011,47,Debris Burning,5.0,2455610.5,2455608.5,49.0,820.0,1500.0,...,AL,2011-02-16,31.442815,201.5,8.048481,0.0,76.942800,49.340830,0.100000,2
2,2,2011,48,Arson,7.0,2455609.5,2455609.5,48.0,2030.0,2300.0,...,AL,2011-02-17,7.572142,100.6,8.064342,0.0,70.000000,33.000000,0.200000,2
3,3,2011,50,Arson,7.0,2455617.5,2455611.5,56.0,1800.0,1500.0,...,AL,2011-02-19,49.942890,182.9,8.037152,0.0,66.000000,49.000000,0.282843,2
4,4,2011,51,Debris Burning,5.0,2455625.5,2455612.5,64.0,1500.0,1900.0,...,AL,2011-02-20,49.942890,189.0,8.042611,0.0,76.657125,49.084371,0.100000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367739,367739,2015,75,Miscellaneous,9.0,NaN,2457097.5,NaN,NaN,NaN,...,VA,2015-03-16,33.568885,283.5,7.366097,0.0,61.000000,39.000000,0.100000,3
367740,367740,2015,95,Miscellaneous,9.0,NaN,2457117.5,NaN,NaN,NaN,...,VA,2015-04-05,33.568885,283.5,7.358285,0.0,57.000000,39.000000,0.100000,4
367741,367741,2015,37,Miscellaneous,9.0,NaN,2457059.5,NaN,NaN,NaN,...,VA,2015-02-06,38.354455,247.8,7.406834,0.0,70.218508,43.916184,0.100000,2
367742,367742,2015,89,Miscellaneous,9.0,NaN,2457111.5,NaN,NaN,NaN,...,VA,2015-03-30,38.354455,208.2,7.384267,0.0,69.862021,43.603630,0.141421,3


In [4]:
df.columns

Index(['Unnamed: 0', 'FIRE_YEAR', 'DISCOVERY_DOY', 'STAT_CAUSE_DESCR',
       'STAT_CAUSE_CODE', 'CONT_DATE', 'DISCOVERY_DATE', 'CONT_DOY',
       'DISCOVERY_TIME', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'LATITUDE_x', 'LONGITUDE_x', 'OWNER_CODE', 'OWNER_DESCR', 'STATE',
       'datetime', 'population_density', 'ELEVATION', 'AWND', 'PRCP', 'TMAX',
       'TMIN', 'distance', 'month'],
      dtype='object')

In [24]:
mean_pd = df[df['population_density']>=0].groupby('STATE')['population_density'].mean().reset_index()
ml_df = df[['month', 'STATE', 'TMAX', 'TMIN', 'AWND']]
ml_df = ml_df.groupby(['month', 'STATE']).agg({'month' : ['count'], 'TMAX' : 'mean', 'TMIN' : 'mean', 'AWND' : 'mean'}).reset_index()
ml_df.columns = [' '.join(col).strip() for col in ml_df.columns.values]
merged = ml_df.merge(mean_pd, how='inner', left_on='STATE', right_on='STATE')
encoding = pd.get_dummies(merged.STATE)
merged = merged.join(encoding)
merged

,month,STATE,month count,TMAX mean,TMIN mean,AWND mean,population_density,AK,AL,AR,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1,AK,2,26.500000,5.500000,10.850000,3.536666,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,AK,3,21.666667,4.666667,14.121259,3.536666,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,AK,19,37.421053,17.052632,8.016115,3.536666,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,AK,145,51.228985,28.280824,8.344330,3.536666,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,AK,771,65.722132,38.080121,8.136232,3.536666,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,7,DE,7,90.797158,69.309613,5.922331,90.880529,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599,8,DE,2,77.024819,56.101038,7.442432,90.880529,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600,9,DE,5,77.768067,56.033829,6.307507,90.880529,0,0,0,...,0,0,0,0,0,0,0,0,0,0
601,10,DE,1,66.000000,60.000000,7.664107,90.880529,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
import autosklearn.regression 
from sklearn.model_selection import train_test_split

In [ ]:
y = merged['month count'].to_frame()
X = merged.drop(['month count', 'STATE'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/content/tmp'
)
automl.fit(X_train, y_train)

In [28]:
import sklearn
train_predictions = automl.predict(X_train)
print("Training R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
train_predictions = automl.predict(X_test)
print("Testing R2 score:", sklearn.metrics.r2_score(y_test, train_predictions))

Training R2 score: 0.9154359884190083
Testing R2 score: 0.8188120723918203


In [29]:
automl.show_models()

"[(0.440000, SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'fast_ica', 'regressor:__choice__': 'gradient_boosting', 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': 0.7049491520337817, 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': 0.23602880130851764, 'feature_preprocessor:fast_ica:algorithm': 'deflation', 'feature_preprocessor:fast_ica:fun': 'cube', 'feature_preprocessor:fast_ica:whiten': 'False', 'regressor:gradient_boosting:early_stop': 'off', 'regressor:gradient_boosting:l2_regularization': 4.696295762197504e-10, 'regressor:gradient_boosting:learning_rate': 0.0326935760

In [31]:
#predict
from sklearn.linear_model import LinearRegression
temp = df[['FIRE_YEAR', 'month', 'STATE', 'AWND', 'TMAX', 'TMIN']]
AWND_predict = temp.groupby(['FIRE_YEAR', 'month', 'STATE']).agg({'AWND' : 'mean'}).reset_index()
encoding = pd.get_dummies(AWND_predict.STATE)
AWND_predict = AWND_predict.join(encoding)
y = AWND_predict['AWND']
x = AWND_predict.drop(['STATE', 'AWND'], axis=1)
AWND_model = LinearRegression().fit(x, y)

In [32]:
TMAX_predict = temp.groupby(['FIRE_YEAR', 'month', 'STATE']).agg({'TMAX' : 'mean'}).reset_index()
encoding = pd.get_dummies(TMAX_predict.STATE)
TMAX_predict = TMAX_predict.join(encoding)
y = TMAX_predict['TMAX']
x = TMAX_predict.drop(['STATE', 'TMAX'], axis=1)
TMAX_model = LinearRegression().fit(x, y)

In [33]:
TMIN_predict = temp.groupby(['FIRE_YEAR', 'month', 'STATE']).agg({'TMIN' : 'mean'}).reset_index()
encoding = pd.get_dummies(TMIN_predict.STATE)
TMIN_predict = TMIN_predict.join(encoding)
y = TMIN_predict['TMIN']
x = TMIN_predict.drop(['STATE', 'TMIN'], axis=1)
TMIN_model = LinearRegression().fit(x, y)

In [34]:
states = set(df['STATE'].to_list())
states = list(states)
states.sort()
year = [2022]
months = list(range(1,13))
year_df = pd.DataFrame(year, columns=['FIRE_YEAR'])
month_df = pd.DataFrame(months, columns=['month_'])
State_df = pd.DataFrame(states, columns=['STATE'])
raw = year_df.merge(month_df, how='cross')
raw = raw.merge(State_df, how='cross')

encoding = pd.get_dummies(raw.STATE)
input_df = raw.join(encoding)
input_df = input_df.drop(['STATE'], axis=1)
input_df

,FIRE_YEAR,month_,AK,AL,AR,AZ,CA,CO,CT,DC,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,2022,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2022,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
608,2022,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
609,2022,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
610,2022,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [35]:
raw

,FIRE_YEAR,month_,STATE
0,2022,1,AK
1,2022,1,AL
2,2022,1,AR
3,2022,1,AZ
4,2022,1,CA
...,...,...,...
607,2022,12,VT
608,2022,12,WA
609,2022,12,WI
610,2022,12,WV


In [36]:
AWND_2022 = AWND_model.predict(input_df)
TMIN_2022 = TMIN_model.predict(input_df)
TMAX_2022 = TMAX_model.predict(input_df)

In [37]:
raw['TMAX_mean'] = TMAX_2022
raw['TMIN_mean'] = TMIN_2022
raw['AWND_mean'] = AWND_2022
mean_pd = df[df['population_density']>=0].groupby('STATE')['population_density'].mean().reset_index()
raw = raw.merge(mean_pd, how='inner', on='STATE')
raw.join(pd.get_dummies(raw.STATE))
input_2022 = raw.join(pd.get_dummies(raw.STATE))
input_2022
input_2022 = input_2022.drop(['STATE', 'FIRE_YEAR'], axis=1)
input_2022

,month_,TMAX_mean,TMIN_mean,AWND_mean,population_density,AK,AL,AR,AZ,CA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1,43.352051,23.789795,7.746155,3.536666,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,43.882812,24.369629,7.723572,3.536666,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,44.413574,24.949219,7.700989,3.536666,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,44.944336,25.529053,7.678406,3.536666,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,45.475098,26.108887,7.655823,3.536666,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,8,64.568848,39.268555,6.976196,3.667393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
608,9,65.099609,39.848389,6.953613,3.667393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
609,10,65.630371,40.427979,6.931030,3.667393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
610,11,66.161133,41.007812,6.908447,3.667393,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
frequency_2022 = automl.predict(input_2022)
raw['frequency'] = frequency_2022
raw['frequency'] = raw['frequency'].apply(lambda x: int(x) if x>=0 else 0)
raw[['FIRE_YEAR', 'month_', 'STATE', 'frequency']].to_csv('frequency_2022.csv')